<a href="https://colab.research.google.com/github/adam-pearman/advent_of_code/blob/main/2025/day_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2025 - Day 10

In [ ]:
!pip install z3-solver

In [4]:
from re import search, findall
from z3 import Bool, Optimize, If, Sum, sat, Int

machines = []
with open('input.txt') as f:
  for line in f:
    line = line.strip()

    diagram = search(r"\[[.#]+\]", line).group()
    buttons = findall(r"\(([0-9,]*)\)", line)
    joltages = search(r"\{([0-9,]+)\}", line).group(1)

    machines.append((diagram, buttons, joltages))

def parse_diagram(diagram):
    return [1 if c == '#' else 0 for c in diagram[1:-1]]

def parse_buttons(buttons):
    return [tuple(map(int, b.split(','))) for b in buttons]

def parse_joltages(joltages):
    return list(map(int, joltages.split(',')))

## Part 1

In [5]:
def part1(diagram, buttons):
    n_lights = len(diagram)
    n_buttons = len(buttons)

    x = [Bool(f"x{i}") for i in range(n_buttons)]
    s = Optimize()

    for light in range(n_lights):
        toggles = [x[i] for i in range(n_buttons) if light in buttons[i]]
        if toggles:
            ints = [If(var, 1, 0) for var in toggles]
            s.add((Sum(ints) % 2) == diagram[light])
        else:
            s.add((0) == diagram[light])

    press_count = Sum([If(var, 1, 0) for var in x])
    s.minimize(press_count)

    if s.check() == sat:
        m = s.model()
        return m.eval(press_count).as_long()
    else:
        return None

total = 0
for diagram, buttons, _ in machines:
    total += part1(parse_diagram(diagram), parse_buttons(buttons))

print(total)

399


## Part 2

In [7]:
def part2(joltages, buttons):
    n_joltages = len(joltages)
    n_buttons = len(buttons)

    x = [Int(f"x{i}") for i in range(n_buttons)]
    s = Optimize()

    for var in x:
        s.add(var >= 0)

    for joltage in range(n_joltages):
        contributors = [x[i] for i in range(n_buttons) if joltage in buttons[i]]
        if contributors:
            s.add(Sum(contributors) == joltages[joltage])
        else:
            s.add(joltages[joltage] == 0)

    press_total = Sum(x)
    s.minimize(press_total)

    if s.check() == sat:
      m = s.model()
      presses = m.eval(press_total).as_long()
      return presses
    else:
      return None

total = 0
for _, buttons, joltages in machines:
    total += part2(parse_joltages(joltages), parse_buttons(buttons))

print(total)

15631
